In [21]:
!pip install accelerate

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoConfig,AutoModelForCausalLM
import pandas as pd

In [3]:
import torch

In [4]:
config = AutoConfig.from_pretrained("neulab/omnitab-large-finetuned-wtq")
# IMPORTANT: the initial BART model's decoding is penalized by no_repeat_ngram_size, and thus
# we should disable it here to avoid problematic generation
config.no_repeat_ngram_size = 3 # В omnitab = 3, part exec = 0
config.max_length = 1024
#config.early_stopping = False #  В омнитаб нет

#model = AutoModel.from_config(config) # Для создания экземпляра модели без предобученных весов
tokenizer = AutoTokenizer.from_pretrained('neulab/omnitab-large-finetuned-wtq')
model = AutoModelForSeq2SeqLM.from_pretrained('neulab/omnitab-large-finetuned-wtq')
#model = AutoModelForCausalLM.from_pretrained("bert-base-uncased", device_map = 'auto')
device = "cuda" if torch.cuda.is_available() else "cpu"

model.to(device)

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [14]:
data = {
    'year': [1896, 1900, 1904, 2004, 2008, 2012],
    'city': ['athens', 'paris', 'st. louis', 'athens', 'beijing', 'london']
}
table = pd.DataFrame.from_dict(data)

query = 'In which year did beijing host the Olympic Games?'
encoding = tokenizer(table=table, query=query, return_tensors='pt')

outputs = model.generate(**encoding)

print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

[' 2008']


In [16]:
next(model.parameters()).device

device(type='cpu')

In [20]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [27]:
from datasets import load_dataset


In [29]:
data_files = {"train":"data/wtq_lf_preorder/train/data-00000-of-00001.arrow",
              "test":"data/wtq_lf_preorder/test/data-00000-of-00001.arrow",
              "validation":"data/wtq_lf_preorder/validation/data-00000-of-00001.arrow"}
datasets = load_dataset("arrow", data_files=data_files)


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [30]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'answers', 'table', 'lf_p', 'lf_pc', 'lf_pcs', 'lf_pcsgbh', 'lf_pcsgbhob', 'lf_pcsgbhoba', 'lf_pcsgbhobaop', 'lf_pcsgbhobaopl', 'sql'],
        num_rows: 8610
    })
    test: Dataset({
        features: ['id', 'question', 'answers', 'table', 'lf_p', 'lf_pc', 'lf_pcs', 'lf_pcsgbh', 'lf_pcsgbhob', 'lf_pcsgbhoba', 'lf_pcsgbhobaop', 'lf_pcsgbhobaopl', 'sql'],
        num_rows: 4344
    })
    validation: Dataset({
        features: ['id', 'question', 'answers', 'table', 'lf_p', 'lf_pc', 'lf_pcs', 'lf_pcsgbh', 'lf_pcsgbhob', 'lf_pcsgbhoba', 'lf_pcsgbhobaop', 'lf_pcsgbhobaopl', 'sql'],
        num_rows: 2092
    })
})

In [31]:
target_col_name = f"lf_pcs"


In [ ]:
features = ["question","table"]
def length_filter(example):
    return {f"length_{target_col_name}": len(tokenizer.tokenize(example[target_col_name]))<=max_target_length}

for mode in ['train',"validation"]:

    if mode =="train":
        datasets[mode] = datasets[mode].map(length_filter)
        filter_col = [idx for idx, i in enumerate(datasets[mode][filter_col_name]) if i==1]
        logger.info(f"remove {len(datasets[mode])-len(filter_col)} observations with length > {max_target_length}.")
        logger.info(f"Previous {mode} size {len(datasets[mode])} actual {mode} size {len(filter_col)}")
        datasets[mode] = datasets[mode].select(filter_col)

    removed_cols = set(datasets[mode].features.keys())-set([target_col_name]+features)
    datasets[mode] = datasets[mode].remove_columns(removed_cols)
    datasets[mode] = datasets[mode].rename_column(target_col_name, 'answers')

In [1]:
from data_processing.metrics import (to_value_list,
                                     to_value,
                                     plot_and_save_model_performance,
                                     fuzzy_matching,
                                     strict_denotation_accuracy,
                                     flexible_denotation_accuracy)

/opt/conda/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


<ipykernel.iostream.OutStream object at 0x7ff016c9e460> Reading dataset from /home/jovyan/cloud/postgraduate/works/Partial-Exec/data/tables/tagged/pristine-seen-tables.tagged
<ipykernel.iostream.OutStream object at 0x7ff016c9e460> Reading dataset from /home/jovyan/cloud/postgraduate/works/Partial-Exec/data/tables/tagged/pristine-unseen-tables.tagged
<ipykernel.iostream.OutStream object at 0x7ff016c9e460> Reading dataset from /home/jovyan/cloud/postgraduate/works/Partial-Exec/data/tables/tagged/training.tagged


In [ ]:
output_file = f'{path_model_folder}/model_evaluation_results.json'
target_col_name = f"lf_{Omega_include.lower()}"
Omega_include = path_model_folder.split('/')[-1].split('_')[1]
datasets = load_from_disk(f"data/wtq_lf_{args.flatten_mode}")


In [ ]:
path_tokenizer_config = "microsoft/tapex-large"
tokenizer = TapexTokenizer.from_pretrained(path_tokenizer_config)
config = AutoConfig.from_pretrained(path_tokenizer_config)
config.no_repeat_ngram_size = 0
config.max_length = 1024
config.early_stopping = False

features = ["question", "table"]

In [ ]:
removed_cols = set(datasets["validation"].features.keys())-set([target_col_name]+features+["answers"])
datasets["validation"] = datasets["validation"].remove_columns(removed_cols)
datasets["validation"] = datasets["validation"].rename_column(target_col_name, 'answers_lf')
datasets["validation"] = datasets["validation"].rename_column("answers", 'answers_exec')
datasets["test"] = datasets["test"].rename_column("answers", 'answers_exec')

In [ ]:
rnd = random.randint(0, len(datasets["validation"]))

In [ ]:
def preprocess_tableqa_function(examples, is_test=False):

        questions = [question.lower() for question in examples["question"]]
        example_tables = examples["table"]
        tables = [
            pd.DataFrame.from_records(example_table["rows"], columns=example_table["header"])
            for example_table in example_tables
        ]
        
        model_inputs = tokenizer(
            table=tables, query=questions, max_length=1024, padding=False, truncation=True
        )
        model_inputs["answers_exec"] = examples["answers_exec"] 
        if not is_test:
            model_inputs["answers_lf"] = examples["answers_lf"]
            
        return model_inputs


In [ ]:
def evaluate_model(model, dataset, comp_graph = False, is_test=False,flatten_mode="preorder"):
        model.eval()
        perfs={ "Strict_Denotation_Accuracy_Execs":[]}
        for idx, example in tqdm(enumerate(dataset), total=len(dataset)):
            print(f"idx {idx}")
            input_ids = example["input_ids"]
            attention_mask = example["attention_mask"]

            input_ids = torch.tensor(input_ids).unsqueeze(0).to(device)
            attention_mask = torch.tensor(attention_mask).unsqueeze(0).to(device)
            generated_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask)
            prediction_lf = tokenizer.decode(generated_ids.squeeze(), skip_special_tokens=True).strip()
            if comp_graph:
                header=tokenizer.decode(input_ids.squeeze(), skip_special_tokens=True)
                header=header.split(' col : ')[1].split(' row 1 : ')[0].split(' |')
                header=[h.strip() for h in header]
                #G=parse(prediction_lf, header, flatten_mode=args.flatten_mode)
                try:
                    print('1')
                    G=parse(prediction_lf, header, flatten_mode=flatten_mode)
                    print('2')
                    prediction_exec = G.executed_last_node()
                    print(f'answers_exec {answers_exec} prediction_exec {prediction_exec}')
                    acc_ex_stric = strict_denotation_accuracy(to_value_list(answers_exec), to_value_list(prediction_exec))

                except Exception as e:
                    print(e)
                    acc_ex_stric = False

            
            else:
                acc_ex_stric = strict_denotation_accuracy(to_value_list(answers_exec), to_value_list(prediction_exec))
            print(f'Strict_Denotation_Accuracy_Execs {acc_ex_stric}')
            perfs["Strict_Denotation_Accuracy_Execs"].append(acc_ex_stric)

        Strict_Denotation_Accuracy_Exec = sum(perfs["Strict_Denotation_Accuracy_Execs"])/len(perfs["Strict_Denotation_Accuracy_Execs"])

        return {'Strict_Denotation_Accuracy_Exec': Strict_Denotation_Accuracy_Exec}  # example
